In [ ]:
import eotdl

eotdl.__version__

Stage the model


In [ ]:
from eotdl.models import stage_model

path = stage_model("RoadSegmentation", path="data", version=2, force=True, assets=True)
model = f"{path}/model.onnx"

model

Prepare the inputs


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

img_path = "assets/deep_globe.jpg"

img = Image.open(img_path)
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
import numpy as np

imgs = np.array(img).transpose(2, 0, 1)[np.newaxis, ...].astype(np.float32) / 255.0

imgs.shape, imgs.dtype, imgs.min(), imgs.max()

Load and run the model


In [ ]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession(model)
input_name = ort_session.get_inputs()[0].name


ort_inputs = {input_name: imgs}
ort_outs = ort_session.run(None, ort_inputs)
mask = ort_outs[0] > 0

Visualize the results


In [ ]:
# Squeeze the mask to remove extra dimensions
mask_squeezed = np.squeeze(mask)

# Create a figure with 3 subplots in a row
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 7))

# Plot the original image
ax1.imshow(img)
ax1.axis("off")
ax1.set_title("Original Image")

# Plot the segmentation mask
ax2.imshow(mask_squeezed, cmap="gray")
ax2.axis("off")
ax2.set_title("Segmentation Mask")

# Plot the image with mask overlay
overlay = np.zeros_like(img)
overlay[:, :, 1] = mask_squeezed * 255  # Green channel for the mask
ax3.imshow(img)
ax3.imshow(overlay, alpha=0.5)
ax3.axis("off")
ax3.set_title("Image with Mask Overlay")

plt.tight_layout()
plt.show()